In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def actor(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('actor', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def generator(actions, state_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=actions, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=state_size)        
        return logits

In [6]:
def discriminator(states, actions, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        return logits

In [7]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    actions_logits = actor(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ###############################################
    next_states_logits = generator(actions=actions_logits, hidden_size=hidden_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    aloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits, 
                                                                    labels=next_states_labels))
    ####################################################
    dQs = discriminator(actions=actions_labels, hidden_size=hidden_size, states=states, action_size=action_size)
    rates = tf.reshape(rates, shape=[-1, 1])
    dloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dQs, # GAN
                                                                   labels=rates)) # 0-1
    ####################################################
    gQs = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states, action_size=action_size, 
                        reuse=True)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.zeros_like(gQs))) # 0-1
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.ones_like(gQs))) # 0-1
    #####################################################
    next_actions_logits = actor(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    gQs2 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states, 
                         action_size=action_size, reuse=True)
    gQs2 = tf.reshape(gQs2, shape=[-1]) * (1-dones)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                    labels=tf.zeros_like(gQs2))) # 0-1
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                     labels=tf.ones_like(gQs2))) # 0-1
    ##################################################### repeatable!
    next_states_logits = generator(actions=next_actions_logits, hidden_size=hidden_size, state_size=state_size, 
                                   reuse=True)
    next_actions_logits = actor(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                                reuse=True)
    gQs3 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states_logits, 
                         action_size=action_size, reuse=True)
    dones2 = tf.concat(axis=0, values=[dones[1:], tf.ones(shape=[1])])
    gQs3 = tf.reshape(gQs3, shape=[-1]) * (1-dones2)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
                                                                    labels=tf.zeros_like(gQs3))) # 0-1
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
                                                                     labels=tf.ones_like(gQs3))) # 0-1
    return actions_logits, aloss, dloss, aloss2

In [8]:
def model_opt(a_loss, a_loss2, d_loss, a_learning_rate, d_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('actor')]
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        d_opt = tf.train.AdamOptimizer(d_learning_rate).minimize(d_loss, var_list=d_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, d_opt, a_opt2

In [9]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, d_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.d_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, #model input 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.d_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        a_loss2=self.a_loss2, 
                                                        d_loss=self.d_loss,
                                                        a_learning_rate=a_learning_rate,
                                                        d_learning_rate=d_learning_rate)

In [10]:
# def sample(buffer, batch_size):
#     idx = np.random.choice(np.arange(len(buffer)), size=batch_size, replace=False)
#     return [buffer[ii] for ii in idx]

In [11]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [12]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [13]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
d_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size: 200/500 a successfull episode size
gamma = 0.99                   # future reward discount

In [14]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              a_learning_rate=a_learning_rate, d_learning_rate=d_learning_rate, gamma=gamma)

# Init the memory
memory = Memory(max_size=memory_size)

In [15]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        state = env.reset()
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
# aloss_list, dloss_list, aloss2_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0 # each episode
        aloss_batch, dloss_batch, aloss2_batch = [], [], []
        state = env.reset() # each episode
        num_step = 0 # each episode

        # Training steps/batches
        while True:
            # Explore (Env) or Exploit (Model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            rate = -1
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Rating the memory
            if done is True:
                rate = total_reward/500 # update rate at the end/ when episode is done
                for idx in range(num_step): # episode length
                    if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                        memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                        
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones)==1 and len(dones) >= 1 and np.max(rates) > 0:
                    # print('np.count_nonzero(dones)==1 and len(dones) >= 1 and np.max(rates) > 0: ', 
                    #       np.count_nonzero(dones), len(dones), np.max(rates))
                    break
            if np.count_nonzero(dones)!=1 and len(dones) < 1 and np.max(rates) <= 0:
                print(np.count_nonzero(dones), len(dones), np.max(rates))
                break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                feed_dict = {model.states: states, 
                                            model.actions: actions,
                                            model.next_states: next_states,
                                            model.rewards: rewards,
                                            model.dones: dones,
                                            model.rates: rates})
            dloss, _ = sess.run([model.d_loss, model.d_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print('dones:', 
            #       len(dones), np.count_nonzero(dones), 
            #       len(dones1), np.count_nonzero(dones1), 
            #       len(dones2), np.count_nonzero(dones2))
            aloss_batch.append(aloss)
            dloss_batch.append(dloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'dloss:{:.4f}'.format(np.mean(dloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        # gloss_list.append([ep, np.mean(gloss_batch)])
        # dloss_list.append([ep, np.mean(dloss_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:19.0000 R:19.0000 rate:0.0380 aloss:1.3942 dloss:2.6111 aloss2:2.1867 exploreP:0.9981
Episode:1 meanR:19.5000 R:20.0000 rate:0.0400 aloss:1.3874 dloss:2.6480 aloss2:2.1640 exploreP:0.9961
Episode:2 meanR:21.6667 R:26.0000 rate:0.0520 aloss:1.3934 dloss:2.6125 aloss2:2.1973 exploreP:0.9936
Episode:3 meanR:24.2500 R:32.0000 rate:0.0640 aloss:1.3916 dloss:2.4914 aloss2:2.2861 exploreP:0.9904
Episode:4 meanR:27.2000 R:39.0000 rate:0.0780 aloss:1.3847 dloss:2.5490 aloss2:2.2595 exploreP:0.9866
Episode:5 meanR:24.5000 R:11.0000 rate:0.0220 aloss:1.3827 dloss:2.6283 aloss2:2.1680 exploreP:0.9856
Episode:6 meanR:23.0000 R:14.0000 rate:0.0280 aloss:1.3811 dloss:2.6298 aloss2:2.1594 exploreP:0.9842
Episode:7 meanR:22.5000 R:19.0000 rate:0.0380 aloss:1.3853 dloss:2.4828 aloss2:2.2995 exploreP:0.9823
Episode:8 meanR:27.6667 R:69.0000 rate:0.1380 aloss:1.3857 dloss:2.4365 aloss2:2.3331 exploreP:0.9757
Episode:9 meanR:26.4000 R:15.0000 rate:0.0300 aloss:1.3924 dloss:2.3739 aloss2:2.3

Episode:80 meanR:23.4691 R:18.0000 rate:0.0360 aloss:1.4128 dloss:1.0026 aloss2:5.4815 exploreP:0.8286
Episode:81 meanR:23.3293 R:12.0000 rate:0.0240 aloss:1.4023 dloss:1.0599 aloss2:5.3990 exploreP:0.8276
Episode:82 meanR:23.2289 R:15.0000 rate:0.0300 aloss:1.3965 dloss:1.0765 aloss2:5.2690 exploreP:0.8264
Episode:83 meanR:23.2262 R:23.0000 rate:0.0460 aloss:1.4067 dloss:1.0498 aloss2:5.7245 exploreP:0.8245
Episode:84 meanR:23.2000 R:21.0000 rate:0.0420 aloss:1.3950 dloss:1.0268 aloss2:5.8001 exploreP:0.8228
Episode:85 meanR:23.2209 R:25.0000 rate:0.0500 aloss:1.4004 dloss:1.0408 aloss2:5.6892 exploreP:0.8208
Episode:86 meanR:23.3563 R:35.0000 rate:0.0700 aloss:1.3972 dloss:1.0085 aloss2:5.8917 exploreP:0.8180
Episode:87 meanR:23.3068 R:19.0000 rate:0.0380 aloss:1.4145 dloss:0.9640 aloss2:6.1187 exploreP:0.8164
Episode:88 meanR:23.5730 R:47.0000 rate:0.0940 aloss:1.4051 dloss:1.0196 aloss2:5.8171 exploreP:0.8126
Episode:89 meanR:23.4556 R:13.0000 rate:0.0260 aloss:1.3956 dloss:0.9314 

Episode:160 meanR:19.7500 R:11.0000 rate:0.0220 aloss:1.3954 dloss:0.7213 aloss2:8.5625 exploreP:0.7154
Episode:161 meanR:19.7900 R:25.0000 rate:0.0500 aloss:1.4018 dloss:0.6831 aloss2:8.7730 exploreP:0.7136
Episode:162 meanR:19.4900 R:12.0000 rate:0.0240 aloss:1.3924 dloss:0.6454 aloss2:8.8754 exploreP:0.7128
Episode:163 meanR:19.5100 R:14.0000 rate:0.0280 aloss:1.3879 dloss:0.7231 aloss2:9.3204 exploreP:0.7118
Episode:164 meanR:19.5500 R:22.0000 rate:0.0440 aloss:1.3794 dloss:0.6855 aloss2:9.0548 exploreP:0.7102
Episode:165 meanR:19.6400 R:26.0000 rate:0.0520 aloss:1.3947 dloss:0.6695 aloss2:8.8175 exploreP:0.7084
Episode:166 meanR:19.1500 R:12.0000 rate:0.0240 aloss:1.3930 dloss:0.6489 aloss2:9.0865 exploreP:0.7076
Episode:167 meanR:19.0600 R:11.0000 rate:0.0220 aloss:1.3854 dloss:0.7208 aloss2:9.1260 exploreP:0.7068
Episode:168 meanR:18.6700 R:9.0000 rate:0.0180 aloss:1.3803 dloss:0.6261 aloss2:8.4877 exploreP:0.7062
Episode:169 meanR:18.7600 R:20.0000 rate:0.0400 aloss:1.3959 dlos

Episode:239 meanR:44.3500 R:66.0000 rate:0.1320 aloss:1.3847 dloss:0.6573 aloss2:9.9675 exploreP:0.4806
Episode:240 meanR:44.4800 R:27.0000 rate:0.0540 aloss:1.3702 dloss:0.6472 aloss2:10.0399 exploreP:0.4793
Episode:241 meanR:45.9900 R:163.0000 rate:0.3260 aloss:1.3785 dloss:0.6561 aloss2:9.9251 exploreP:0.4717
Episode:242 meanR:46.7400 R:88.0000 rate:0.1760 aloss:1.3981 dloss:0.6218 aloss2:9.7745 exploreP:0.4677
Episode:243 meanR:47.2200 R:60.0000 rate:0.1200 aloss:1.3951 dloss:0.6715 aloss2:9.8351 exploreP:0.4649
Episode:244 meanR:47.4900 R:46.0000 rate:0.0920 aloss:1.4051 dloss:0.6369 aloss2:9.8949 exploreP:0.4628
Episode:245 meanR:47.6400 R:24.0000 rate:0.0480 aloss:1.4411 dloss:0.6372 aloss2:9.7782 exploreP:0.4617
Episode:246 meanR:47.7500 R:26.0000 rate:0.0520 aloss:1.3861 dloss:0.7000 aloss2:9.8245 exploreP:0.4606
Episode:247 meanR:47.8200 R:21.0000 rate:0.0420 aloss:1.4093 dloss:0.5934 aloss2:9.7884 exploreP:0.4596
Episode:248 meanR:48.2800 R:89.0000 rate:0.1780 aloss:1.3879 d

Episode:318 meanR:83.2600 R:500.0000 rate:1.0000 aloss:1.4579 dloss:0.6781 aloss2:9.5088 exploreP:0.2429
Episode:319 meanR:83.8200 R:123.0000 rate:0.2460 aloss:1.4514 dloss:0.6714 aloss2:9.5957 exploreP:0.2400
Episode:320 meanR:84.0800 R:149.0000 rate:0.2980 aloss:1.4263 dloss:0.7007 aloss2:9.6151 exploreP:0.2366
Episode:321 meanR:85.5900 R:248.0000 rate:0.4960 aloss:1.4403 dloss:0.7205 aloss2:9.5361 exploreP:0.2311
Episode:322 meanR:87.0500 R:222.0000 rate:0.4440 aloss:1.4520 dloss:0.6934 aloss2:9.4819 exploreP:0.2262
Episode:323 meanR:89.3800 R:272.0000 rate:0.5440 aloss:1.4721 dloss:0.7192 aloss2:9.4781 exploreP:0.2204
Episode:324 meanR:90.7400 R:157.0000 rate:0.3140 aloss:1.4451 dloss:0.7188 aloss2:9.4199 exploreP:0.2171
Episode:325 meanR:92.2400 R:176.0000 rate:0.3520 aloss:1.4437 dloss:0.7538 aloss2:9.3650 exploreP:0.2135
Episode:326 meanR:92.4000 R:147.0000 rate:0.2940 aloss:1.4550 dloss:0.7241 aloss2:9.3546 exploreP:0.2105
Episode:327 meanR:93.3900 R:147.0000 rate:0.2940 aloss:

...

Episode:864 meanR:317.9400 R:341.0000 rate:0.6820 aloss:1.1354 dloss:1.1621 aloss2:13.6596 exploreP:0.0100
Episode:865 meanR:316.7900 R:64.0000 rate:0.1280 aloss:1.1377 dloss:1.1658 aloss2:13.9263 exploreP:0.0100
Episode:866 meanR:312.7700 R:98.0000 rate:0.1960 aloss:1.1580 dloss:1.1721 aloss2:12.2257 exploreP:0.0100
Episode:867 meanR:309.5800 R:181.0000 rate:0.3620 aloss:1.1261 dloss:1.1476 aloss2:13.5581 exploreP:0.0100
Episode:868 meanR:306.7200 R:212.0000 rate:0.4240 aloss:1.1212 dloss:1.1288 aloss2:15.2503 exploreP:0.0100
Episode:869 meanR:301.9600 R:24.0000 rate:0.0480 aloss:1.1137 dloss:1.1617 aloss2:12.5072 exploreP:0.0100
Episode:870 meanR:305.5800 R:500.0000 rate:1.0000 aloss:1.1244 dloss:1.1251 aloss2:16.0250 exploreP:0.0100
Episode:871 meanR:310.4200 R:500.0000 rate:1.0000 aloss:1.1259 dloss:1.1181 aloss2:14.5282 exploreP:0.0100
Episode:872 meanR:315.1800 R:500.0000 rate:1.0000 aloss:1.1251 dloss:1.1183 aloss2:15.2876 exploreP:0.0100
Episode:873 meanR:314.6800 R:72.0000 rat

Episode:941 meanR:240.0400 R:54.0000 rate:0.1080 aloss:1.1040 dloss:0.6739 aloss2:35.3259 exploreP:0.0100
Episode:942 meanR:237.0800 R:134.0000 rate:0.2680 aloss:1.1227 dloss:0.7509 aloss2:21.7805 exploreP:0.0100
Episode:943 meanR:235.9900 R:276.0000 rate:0.5520 aloss:1.1466 dloss:0.7304 aloss2:22.1750 exploreP:0.0100
Episode:944 meanR:237.5900 R:223.0000 rate:0.4460 aloss:1.1311 dloss:0.7308 aloss2:20.7252 exploreP:0.0100
Episode:945 meanR:240.1300 R:323.0000 rate:0.6460 aloss:1.1155 dloss:0.7585 aloss2:22.4697 exploreP:0.0100
Episode:946 meanR:235.9200 R:79.0000 rate:0.1580 aloss:1.1284 dloss:0.7209 aloss2:21.4745 exploreP:0.0100
Episode:947 meanR:232.7700 R:27.0000 rate:0.0540 aloss:1.1132 dloss:0.7475 aloss2:21.5536 exploreP:0.0100
Episode:948 meanR:232.1500 R:438.0000 rate:0.8760 aloss:1.1511 dloss:0.7384 aloss2:22.4193 exploreP:0.0100
Episode:949 meanR:229.5200 R:23.0000 rate:0.0460 aloss:1.1689 dloss:0.7388 aloss2:20.8859 exploreP:0.0100
Episode:950 meanR:228.5300 R:298.0000 rat

Episode:1018 meanR:242.2100 R:500.0000 rate:1.0000 aloss:1.1239 dloss:0.8331 aloss2:18.7519 exploreP:0.0100
Episode:1019 meanR:240.4900 R:38.0000 rate:0.0760 aloss:1.1347 dloss:0.8398 aloss2:19.4044 exploreP:0.0100
Episode:1020 meanR:239.5000 R:37.0000 rate:0.0740 aloss:1.1141 dloss:0.7798 aloss2:18.3922 exploreP:0.0100
Episode:1021 meanR:237.2100 R:271.0000 rate:0.5420 aloss:1.1342 dloss:0.8114 aloss2:18.9709 exploreP:0.0100
Episode:1022 meanR:237.2100 R:500.0000 rate:1.0000 aloss:1.1285 dloss:0.8210 aloss2:19.3859 exploreP:0.0100
Episode:1023 meanR:241.5900 R:500.0000 rate:1.0000 aloss:1.1282 dloss:0.8217 aloss2:19.1562 exploreP:0.0100
Episode:1024 meanR:241.9900 R:219.0000 rate:0.4380 aloss:1.1366 dloss:0.8361 aloss2:19.1916 exploreP:0.0100
Episode:1025 meanR:242.2400 R:40.0000 rate:0.0800 aloss:1.1098 dloss:0.7953 aloss2:18.6180 exploreP:0.0100
Episode:1026 meanR:242.5500 R:86.0000 rate:0.1720 aloss:1.1269 dloss:0.8819 aloss2:18.6678 exploreP:0.0100
Episode:1027 meanR:247.1600 R:50

Episode:1095 meanR:270.1900 R:500.0000 rate:1.0000 aloss:1.1043 dloss:0.8165 aloss2:21.5018 exploreP:0.0100
Episode:1096 meanR:267.7600 R:23.0000 rate:0.0460 aloss:1.0909 dloss:0.8364 aloss2:21.3056 exploreP:0.0100
Episode:1097 meanR:266.7900 R:231.0000 rate:0.4620 aloss:1.1034 dloss:0.8047 aloss2:21.0114 exploreP:0.0100
Episode:1098 meanR:266.7900 R:500.0000 rate:1.0000 aloss:1.1006 dloss:0.8266 aloss2:21.3696 exploreP:0.0100
Episode:1099 meanR:265.1700 R:22.0000 rate:0.0440 aloss:1.1114 dloss:0.8524 aloss2:20.2676 exploreP:0.0100
Episode:1100 meanR:265.3200 R:50.0000 rate:0.1000 aloss:1.1174 dloss:0.8381 aloss2:21.4674 exploreP:0.0100
Episode:1101 meanR:267.8200 R:285.0000 rate:0.5700 aloss:1.1030 dloss:0.8370 aloss2:21.1050 exploreP:0.0100
Episode:1102 meanR:263.4800 R:66.0000 rate:0.1320 aloss:1.1047 dloss:0.8184 aloss2:21.0025 exploreP:0.0100
Episode:1103 meanR:263.4000 R:63.0000 rate:0.1260 aloss:1.0967 dloss:0.8245 aloss2:20.5769 exploreP:0.0100
Episode:1104 meanR:263.4000 R:500

Episode:1172 meanR:245.7500 R:16.0000 rate:0.0320 aloss:1.1242 dloss:0.8136 aloss2:21.6909 exploreP:0.0100
Episode:1173 meanR:245.7500 R:500.0000 rate:1.0000 aloss:1.1071 dloss:0.7878 aloss2:22.5969 exploreP:0.0100
Episode:1174 meanR:250.3700 R:500.0000 rate:1.0000 aloss:1.1047 dloss:0.7955 aloss2:22.1576 exploreP:0.0100
Episode:1175 meanR:250.3700 R:500.0000 rate:1.0000 aloss:1.1061 dloss:0.8044 aloss2:23.0464 exploreP:0.0100
Episode:1176 meanR:245.5500 R:18.0000 rate:0.0360 aloss:1.1207 dloss:0.8094 aloss2:22.4593 exploreP:0.0100
Episode:1177 meanR:245.5500 R:500.0000 rate:1.0000 aloss:1.1060 dloss:0.7874 aloss2:22.4259 exploreP:0.0100
Episode:1178 meanR:242.8300 R:228.0000 rate:0.4560 aloss:1.1015 dloss:0.7842 aloss2:22.8720 exploreP:0.0100
Episode:1179 meanR:240.5200 R:40.0000 rate:0.0800 aloss:1.1151 dloss:0.8050 aloss2:23.0592 exploreP:0.0100
Episode:1180 meanR:239.8400 R:432.0000 rate:0.8640 aloss:1.1071 dloss:0.8207 aloss2:22.8806 exploreP:0.0100
Episode:1181 meanR:239.8200 R:2

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
# eps, arr = np.array(dloss_list).T
# smoothed_arr = running_mean(arr, 10)
# plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
# plt.plot(eps, arr, color='grey', alpha=0.3)
# plt.xlabel('Episode')
# plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 481.0
total_reward: 481.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.